In [ ]:
from blindNegex import read_negations, process_text

# Example usage
negations_file = '../data/negation_speculation_word.txt'
text_file = '../data/sentences.txt'
output_file = '../data/output_bilnd_negex.txt'

negations = read_negations(negations_file)
process_text(text_file, output_file, negations)